##### Spinel grain segmentaion workflow
### Author: J.F. Einsle 
#### Contact: jfe26@cam.ac.uk



This is a basic workflow that I put together to segment spinel grains from a background matrix. It has been demonstrated to be fairly robust procedure against a decent number of the data sets taken so far.  However, the exact values for thresholds and heights may need to be adjusted.  This can be addreesed in future versions of the notebook.  Also, need to add in saving off images at several of the steps.


### References:
• useful website for python based segmentation: http://emmanuelle.github.io/a-tutorial-on-segmentation.html

• 'Microscope Image Processing' by Q. Wu, F. A. Merchant and K. R. Castleman 

• Matlab / Mathworks blog

• the Scikit-Image examples Gallery: http://scikit-image.org/docs/stable/auto_examples/

#### Note: ususally run with in  %matplotlib qt5, as gives the figures in a new seperate window. this is needed for the interactive tools.  For documenting here going to turn on the %matplotlib inline option.

### Note after hyperspy release 1.3(?) use matplotlib qt5....

on windows this means need to launch from the'conda promt' not just cmd promt.  also note if using the 'easy windows install of hyperspy' then you are still using qt4...makes perfect sense.

## Work Flow / index <a id='link1'></a>

#### <a href='#link2'> **0. Preliminary libaries**</a>

#### <a href='#link3'> **1. Import data, examine raw histogram**</a>

#### <a href='#link4'> **2. Noise filtering and smoothing with total variation filter**</a>

#### <a href='#link5'> **3. Image segmentation: Foreground from background**</a>

#### <a href='#link6'> **4. Image segmentation: pulling apart objects**</a>
#####  <a href='#link7'> **4.1 Generate marker field and distance map **<a/>
##### <a href='#link8'> **4.2 Application of h-maxima **<a/>
#####  <a href='#link9'> **4.3 Boundery removal and overlay**<a/>



#### <a href='#link10'> **5. Manual Seeding**</a>


## 0. Prelimnary Python Libaries <a id='link2'></a>
 <a href='#link2'> **Return to workflow**</a>

In [1]:
%matplotlib qt5
#%matplotlib inline
import hyperspy.api as hs
import numpy as np

import matplotlib.pyplot as plt

import scipy
from scipy import ndimage as ndi

import skimage as ski
from skimage.segmentation import random_walker
from skimage.feature import peak_local_max
from skimage import exposure
from skimage import measure
from skimage import morphology as mph
from skimage import restoration
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray

 ## Load the raw data
 <a id='link3'></a>
 
 <a href='#link2'> **Return to workflow**</a> 

In [2]:
grains=hs.load('/Users/mq20197500/Documents/Spinel_paper_fig/006_grey.tif')

In [3]:
grains 

<Signal2D, title: , dimensions: (|1024, 768)>

In [4]:
grains.plot()

### Examine the histogram

There is a reasonalbe amount of noise in the image above (not un workable,just not making the task of segmentation easy).  So in order to figure out the next step need to examine the distribution of the pixel intesities.  Do this with the histogram below.

#### normalise the data

need to normalise the data and convert to a floating point so that the filters are happy.

In [5]:
grain_nrm = grains.deepcopy()
grain_nrm.data = grain_nrm.data /(np.max(grain_nrm.data))


In [6]:
grain_nrm = grains.deepcopy()
grain_nrm.data = grain_nrm.data /(np.max(grain_nrm.data))

hist = exposure.histogram(grain_nrm.data)


plt.figure()
plt.plot(hist[1], hist[0], label='Raw BSE intesity')
#plt.xlim(0.05,1)
#plt.ylim(0,1100)
plt.legend()
plt.title('Histogram of voxel values')


Text(0.5, 1.0, 'Histogram of voxel values')

 <a href='#link2'> **Return to workflow**</a> 



## Noise filtering / smoothing with total variation filter  <a id='link4'></a>

Segmentation of any data set is improved by applying a filter to smooth out the data, and minimise the noise or grain in the image.  I have generally found for a wide range of electron microscopy data that a total variation filter {see refs on the funtion here: hyperlink}.  This filter has the effect of 'cartooning' or aplifying edge contrast.  From documentation, the best choise for the weight of this filter would be the  the full width at half maxium (FWHM)of the peak associated with the feature of interest.  So examiing the histogram above for the peak centered on ~170, get a FWHM of roughly 30.  However, sometime you need to go a little stronger or weaker to determine the best weight. Here found optimal value at 40. 

#### Aside:
as the data is still only 8 bit the pixel intesity goes from 0-255, if acquired 16 bit or converted to float will get FWHM values that are more "decimal" in nature.

as above use a normalised version of the data.  also for tva or nlm denoiseing the weight or h values are usually based on the measure of noise in the image. a good starting place is the Gaussian noise measuere caputered in the 'estimate_sigma' function

In [9]:
grain_nrm.change_dtype('float32')
sigma_est = np.mean(restoration.estimate_sigma(nlm.data, multichannel=False))
print(sigma_est)

0.020751056319657627


In [10]:
np.min(grain_nrm.data)

0.007843138

In [11]:


tva2 = grain_nrm.deepcopy()
tva2.data =restoration.denoise_tv_chambolle(grain_nrm.data, weight=2*sigma_est)
tva2.metadata.General.title='Total variation filter weight = sigma_est'
tva2.change_dtype('float32')
#tva2.plot(cmap='viridis')
#tva2.save('E17-059_006_tva_grains_sig_est.tif')

## new error

not sure why getting this run time warning.... especially as the task is striaght forward.....

## Non-Local Means Denoiseing

new implimentation of this filter, makes it slightly more `realistic' than the TV filter.....

In [12]:
nlm=grain_nrm.deepcopy()
nlm.data = restoration.denoise_nl_means(grain_nrm.data, h=1.15 * sigma_est, fast_mode=False)
#nlm.plot()
#nlm.change_dtype('float32')
#nlm.save('E17_059_006_nlm_filter.tif')

In [13]:
nlm.plot()
nlm.save('E17_059_006_nlm_filter.tif')

Overwrite 'E17_059_006_nlm_filter.tif' (y/n)?
n


### Compare the two data sets visually

using another Look Up Table (LUT) / color map to plot the data can bring out feature not apparent in grayscale. Also it is just more visually interesting to look at. For instantce the 'magma' color map as it is perceptually uniform, but helps drive contrast a little more than a straight greyscale.  Preceptually uniform colormaps do not give a) false interpitations due to the map being set up non-linerally, and b) will usually print on a grayscale printer correctly.

#### Aside on color
Also changeing the colormap in the script does nothing to how the image will be saved... in that a grayscale image (8/ 16 or 32 bit) will still save as a graysacle tiff. To save the ploted image as a 'color' image you need to convert the information to RGB or HSE data and then save that way. (outside the scope of what I am doing here).  Though if you save the ploted 'figures' these can preserve the color information.


In [14]:
grains.plot(cmap='magma')

tva2.plot(cmap='magma')

### Examine the histogram of the filtered image

in the filtered image histogram can see that the noise is now significantly reduced. also the 'bright' peak associated with the spinels has become narrower.

I also compute and report the Otsu threshold of the image.  This is a classical segmentation tool for splitting image data which is bimodal. Though as will discuss below this does not fully capture all the data we are interested in.

In [14]:
phase_otsu = ski.filters.threshold_otsu(nlm.data)
print('phase_otsu=', phase_otsu)

hist2 = exposure.histogram(nlm.data)


plt.figure()
plt.plot(hist2[1], hist2[0], label=' total variation intesity')
#plt.plot(hist2[1],phase_otsu)
#plt.xlim(0.05,1)
#plt.ylim(0,1100)
plt.legend()
plt.title('Histogram of pixel values')




phase_otsu= 0.3818397674458538


Text(0.5, 1.0, 'Histogram of pixel values')

In [38]:
phase_otsu = ski.filters.threshold_otsu(nlm.data)
print('phase_otsu=', phase_otsu)
seed_value=(1.1*phase_otsu)
 
 
hist2 = exposure.histogram(nlm.data)
 
 
plt.figure()
plt.plot(hist2[1], hist2[0], label=' total variation intesity')
#add Otsu threshold
plt.axvline(phase_otsu, ls='--', lw=3)
plt.text(x=phase_otsu-0.10, y=10000, s='Otsu')
#add Seed value lin
plt.axvline(seed_value, ls='solid', lw=3)
plt.text(x=seed_value+0.05, y=0.85, s='Seed')
plt.title('Histogram of pixel values')

phase_otsu= 0.3818397674458538


/Users/mq20197500/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


Text(0.5, 1.0, 'Histogram of pixel values')

In [16]:
nlm.plot(cmap='viridis')

In [17]:
nlm=hs.load('E17_059_006_nlm_filter.tif')

 <a href='#link2'> **Return to workflow**</a> 


## Image segmentation: pulling apart objects  <a id='link5'></a>

Segmentation for this data has two tasks. First identify the spinel grains from the back ground. The second task is to pull touching grians apart from one another.   

By the histogram above the data has a nice bimodal distribution. In theory the Otsu method of thresholding should segment the grains from the back ground. Regretabily this tends to overselect bright features in this image and creates problems for pulling apart individual grains in the next step. 

In stead will use a watershed critera to define basins of features. The basins here are defined by the pockets created when looking at a height map of the image. Then by seeding the image with areas that define the foreground (ie grains) but not worrying about getting all the grains, and seeding values for the background, I am able to let the watershed algorithm do the work of defining the two regions.

For the height map I use a Scharr filter, which is a variation on the Sobel, that by finding the gradent in the images idetifies edges. 

The seed, or markers, for foreground is roughly the maximal bright peak value. The seed for the background is the peak of the dark peak.


In [18]:
scharr = ski.filters.sobel(nlm.data)

#plt.figure()
#plt.imshow(scharr)

hist3 = exposure.histogram(scharr)


plt.figure()
plt.plot(hist3[1], hist3[0], label='scharr intesity')
#plt.xlim(0.05,1)
#plt.ylim(0,1100)
plt.legend()
plt.title('Histogram of pixel values')


Text(0.5, 1.0, 'Histogram of pixel values')

In [19]:
nlm.plot()

In [20]:
plt.figure()
plt.imshow(scharr,cmap='BuGn')

Here using a watershed transformation to pull the grains out from the background based on the markers set by the histogram of the denoised image.

Markers act as seeds for the watershed.  now to remoive the noisy one need to 

In [21]:
markers = nlm.deepcopy()
markers.data[nlm.data < phase_otsu] = 1
#markers[tva2.data < 0.01] += 1
markers.data[nlm.data > 1.1*phase_otsu] = 2
plt.figure()
plt.imshow(markers)


In [22]:

labels = mph.watershed(scharr, markers.data)
grains_ws= hs.signals.Signal2D(labels)
#grains_ws.change_dtype('float32')
grains_ws.metadata.General.title = 'scharr based water shed'
grains_ws.data[grains_ws.data==1]=0
grains_ws.data[grains_ws.data==2]=1

#grains_ws.data[grains_ws.data==0]=np.nan
grains_ws.plot(cmap='magma', scalebar=False)
num_grains = measure.label(labels, 
                        background=0)
print(num_grains.max())
num_grains = (num_grains + 1).astype(np.uint8)

181


In [23]:
1.2*phase_otsu

0.45820772093502454

In [24]:
grains_ws.plot(cmap='magma', scalebar=False)


next bit here removes 'holes' in the grains which will mess up the watershed segmentation.  This is not always needed.

In [25]:
minsize=300
grain_bin2=grains_ws.deepcopy()
#grain_bin2.data = mph.opening(grain_bin2.data, mph.disk(5))
grain_bin2.data = mph.remove_small_holes(grains_ws.data, area_threshold=minsize)


grain_bin2.plot()

/Users/mq20197500/opt/anaconda3/lib/python3.7/site-packages/skimage/morphology/misc.py:208: UserWarning: Any labeled images will be returned as a boolean array. Did you mean to use a boolean array?
  "Did you mean to use a boolean array?", UserWarning)


save off watersed of all high intesity grains (inclusding saturating metal grains) from background matrix.

In [26]:
grain_bin2.change_dtype('float32')
grain_bin2.save('E17-059_006_nlm__bin_WS_bkg.tif')

Overwrite 'E17-059_006_nlm__bin_WS_bkg.tif' (y/n)?
n


### Remove high intesity data

go back to top of to review histogram (i.e. rerun cell if closed window)



In [27]:
met_markers = nlm.deepcopy()
met_markers.data[nlm.data >= 0.98] =1
#markers[tva2.data < 0.01] += 1
met_markers.data[nlm.data <0.98] =0
met_markers.data = mph.dilation(met_markers.data, mph.disk(5))
#plt.figure()
#plt.imshow(met_markers)
met_markers.plot()
grains3= grain_bin2 - met_markers
grains3.data[grains3.data<0]=0
grains3.save('E17-059_006_nlm__bin_WS_bkg_sat_rmv.tif')
grains3.plot()

Overwrite 'E17-059_006_nlm__bin_WS_bkg_sat_rmv.tif' (y/n)?
n


In [28]:
grains3.change_dtype('uint8')
grains3.save('E17-059_006_nlm__bin_WS_bkg_sat_rmv.tif')


Overwrite 'E17-059_006_nlm__bin_WS_bkg_sat_rmv.tif' (y/n)?
n


In [29]:
minsize=300
grains3.change_dtype('uint8')

grain4=grains3.deepcopy()
#grain_bin2.data = mph.opening(grain_bin2.data, mph.disk(5))
grain4.data = mph.remove_small_holes(grains3.data, area_threshold=minsize)


grain4.plot()



/Users/mq20197500/opt/anaconda3/lib/python3.7/site-packages/skimage/morphology/misc.py:208: UserWarning: Any labeled images will be returned as a boolean array. Did you mean to use a boolean array?
  "Did you mean to use a boolean array?", UserWarning)


In [30]:
grains3=grain4.deepcopy()

grains3.save('E17-059_006_nlm__bin_WS_bkg_sat_rmv.tif')


Overwrite 'E17-059_006_nlm__bin_WS_bkg_sat_rmv.tif' (y/n)?
n


lets try manual seeding
#### <a href='#link10'> **5. Manual Seeding**</a>



 <a href='#link2'> **Return to workflow**</a> 


## Image segmentation: pulling apart objects <a id='link6'></a>

As can be seen in the final foreground - background segmented image, there are several grains that when compared to the grayscale image are now merged into a continious object. This problem is the classical watershed application.(https://en.wikipedia.org/wiki/Watershed_%28image_processing%29)

### Generate marker field and distance map<a id='link7'></a>
To address this we look at the grains isolated from the backgroud (so use the above image as mask to start with) and then figure out how to pull these grains apart. The basic approach to this is find the eulcidian distance of every forground pixel from its neighbors. By finding these maximia and useing them as the seeds combined with the distance map the watershed algorithm cuts apart touching grains. and documentation here: http://scikit-image.org/docs/dev/auto_examples/segmentation/plot_watershed.html#sphx-glr-auto-examples-segmentation-plot-watershed-py.)

#### NOTE

on the distance map perfrom on the binary of the grains where holes have been filled in (grain_bin2) not the inital watershed....

In [31]:
distance = ndi.distance_transform_edt(grains3.data)
hist4 = exposure.histogram(distance)


dist_map=hs.signals.Signal2D(distance)
dist_map.plot()
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)),labels=grain_bin2.data)
loc_max=hs.signals.Signal2D(local_maxi)
#loc_max.plot()

plt.figure()
plt.plot(hist4[1], hist4[0], label='distance = basin depth')
#plt.xlim(0.05,1)
#plt.ylim(0,1100)
plt.legend()
plt.title('Histogram of pixel values')

Text(0.5, 1.0, 'Histogram of pixel values')

In [29]:
dist_map.plot(cmap='BuGn')


In [32]:
plt.figure()
plt.plot(hist4[1], hist4[0], label='distance = basin depth')
#plt.xlim(0.05,1)
#plt.ylim(0,1100)
plt.legend()
plt.title('Histogram of pixel values')

Text(0.5, 1.0, 'Histogram of pixel values')

 <a href='#link2'> **Return to workflow**</a> 

### h-mnima to preserve texture <a id='link8'></a>

Following the worked example in the link above on the data in this notebook results in regions of oversegmentation. This is a well known point of discussion with many of the segmentation algorithms.  One way to address oversegmentation in watershed transformations '... is to modify the image to remove minima that are too shallow. That is exactly what the h-minima transform (imhmin) does.' (https://uk.mathworks.com/company/newsletters/articles/the-watershed-transform-strategies-for-image-segmentation.html')

This function is summarised / demonstrated in 'Microscope Image Processing' by Q. Wu, F. A. Merchant and K. R. Castleman Chapter 8, but the equation describing it is not compleate.  There is a reference where the function is fully defined, but I have not been able to locate it.  Regradless, application of the h-maxima to the positive distance map, or h-minima to the negative distance map (depends how you want to think about the watershed process) results in an image segmentation that manages to isolate the majority of the grains in the image.

In [37]:
max_height = 4
#use hminima fctn as ust upgraded to scikit img 0.13

h_max= mph.h_maxima(distance, h= max_height) 
h_max=hs.signals.Signal2D(h_max)
h_max.data[h_max.data<0]=0
h_max.plot(cmap='magma')

#local_maxi = peak_local_max(h_max.data, indices=False, footprint=np.ones((3, 3)),labels=grain_bin2.data)
#loc_max=hs.signals.Signal2D(local_maxi)
#loc_max.plot()
joining= mph.dilation(h_max.data, mph.disk(3))
seeds = ndi.label(joining)[0]
#markers = mph.opening(markers, mph.disk(3))
#markers = morphology.erosion(markers, morphology.disk(3))
mark_map=hs.signals.Signal2D(seeds)
mark_map.metadata.General.title = 'seeds'

mark_map.change_dtype('float32')
mark_map.data[mark_map.data==0]=np.nan
mark_map.plot(cmap='Reds')
print(np.max(markers))


labels = mph.watershed(-distance, seeds, mask=grains3.data)
grains_sep= hs.signals.Signal2D(labels)
grains_sep.change_dtype('float32')
grains_sep.metadata.General.title = 'h_maxima water shed'
grains_sep.data[grains_sep.data==0]=np.nan
grains_sep.plot(cmap='magma', scalebar=False)
num_grains = measure.label(labels, 
                        background=0)
print(num_grains.max())
num_grains = (num_grains + 1).astype(np.uint8)


/Users/mq20197500/opt/anaconda3/lib/python3.7/site-packages/hyperspy/drawing/utils.py:133: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(**kwargs)
/Users/mq20197500/opt/anaconda3/lib/python3.7/site-packages/hyperspy/drawing/utils.py:133: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(**kwargs)


<Signal2D, title: , dimensions: (|1024, 768)>
281


/Users/mq20197500/opt/anaconda3/lib/python3.7/site-packages/hyperspy/drawing/utils.py:133: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(**kwargs)
/Users/mq20197500/opt/anaconda3/lib/python3.7/site-packages/matplotlib/ticker.py:632: UserWarning: Warning: converting a masked element to nan.
  return '%-12g' % value


save of the grains that have now been pulled apart (and labeled)

In [34]:
grains_sep.save('SEM_img_016_nlm__bin_WS_grains_no_sat_rev4.tif')

 <a href='#link2'> **Return to workflow**</a> 

### Boundery removal and overlay <a id='link9'></a>

This last step cuts off a 4 pixel boundery around every labeled object. This means a) you caneasily tell where one grain is and b) is useing something like ImageJ with Bone J for particle analysis you will easily be able to provide the resutlsing binary for measurement.

The final step is purly for presintation, producing the 'stained glass' look where each labelded particle is trasparently overlayed onto a reference image (in this case the denoised version of the orignal.

#### <a href='#link10'> **5. Manual Seeding**</a>

In [35]:
grain_bd=ski.segmentation.find_boundaries(labels)
grain_bd = mph.dilation(grain_bd,mph.disk(1))
grain_bd= np.invert(grain_bd)
grain_bd=hs.signals.Signal2D(grain_bd)
#grain_bd.plot(cmap='magma')
isolated = grains_ws.deepcopy()
isolated.data= labels  * grain_bd.data
#isolated.data[isolated.data==0]=np.nan

isolated.plot(cmap='magma', scalebar=False)


plt.figure()
plt.imshow(ski.color.label2rgb(isolated.data, nlm.data, kind='overlay', bg_label=0))

ax = plt.axis('off')

/Users/Isra/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [36]:
grain_bd.plot(cmap='magma')

/Users/Isra/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [37]:
nlm.plot(scalebar=False)

/Users/Isra/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [38]:
grains_sep.plot(cmap='magma')

/Users/Isra/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [39]:
isolated.plot(cmap='magma', scalebar=False)


/Users/Isra/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
Traceback (most recent call last):
  File "/Users/Isra/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 388, in process
    proxy(*args, **kwargs)
  File "/Users/Isra/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/__init__.py", line 228, in __call__
    return mtd(*args, **kwargs)
  File "/Users/Isra/anaconda3/lib/python3.6/site-packages/hyperspy/drawing/utils.py", line 154, in function_wrapper
    function()
  File "/Users/Isra/anaconda3/lib/python3.6/site-packages/hyperspy/drawing/figure.py", line 102, in _on_close
    self.events.closed.trigger(obj=self)
  File "<string>", lin

 <a href='#link2'> **Return to workflow**</a> 

## lets look at some particle metrics

In [40]:
num_grains = measure.label(labels, 
                        background=0)
print(num_grains.max())
num_grains = (num_grains + 1).astype(np.uint8)


590


In [43]:
grain_metrics=measure.regionprops(labels)
areas = np.array([prop.area for prop in grain_metrics])
eccs = np.array([prop.eccentricity for prop in grain_metrics])
lab = np.array([prop.label for prop in grain_metrics])
corrs = np.array([prop.coords for prop in grain_metrics])  
equi_dia= np.array([prop.equivalent_diameter for prop in grain_metrics])

maj_ax=np.array([ prop.major_axis_length for prop in grain_metrics])
min_ax=np.array([ prop.minor_axis_length for prop in grain_metrics])
    


/Users/Isra/anaconda3/lib/python3.6/site-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/Users/Isra/anaconda3/lib/python3.6/site-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)


In [44]:
num_bins = len(maj_ax)
counts, bin_edges = np.histogram (np.log10(maj_ax), bins=num_bins, normed=True)
cdf = np.cumsum (counts)
plt.figure()
plt.plot (bin_edges[1:], cdf/cdf[-1])


/Users/Isra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  


In [45]:
hist, bin_edges = np.histogram(maj_ax, bins=20,normed=True)
plt.figure()
plt.bar(bin_edges[:-1], hist, width=5)
plt.show()

/Users/Isra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  """Entry point for launching an IPython kernel.


## Dataframe for pixel units



In [60]:
metrics_list = [ areas, eccs, maj_ax, min_ax ]

In [61]:

metrics_list = [[j[i] for j in metrics_list] for i in range(len(metrics_list[0]))]


In [62]:
mets = np.array(metrics_list)
mets.shape

(590, 4)

In [66]:
met_titles=['Areas', 'eccentricty','Major axis', 'Minor Axis']

In [64]:
import pandas as pd

In [67]:
particle_df = pd.DataFrame.from_records(metrics_list, columns=met_titles)
particle_df

,Areas,eccentricty,Major axis,Minor Axis
0,103,0.760783,15.424582,10.010659
1,656,0.810951,40.508854,23.702279
2,406,0.946743,42.421221,13.659238
3,313,0.781658,28.029960,17.482487
4,874,0.846164,47.986259,25.572958
5,1070,0.896398,59.956806,26.575823
6,80,0.964328,23.642500,6.258371
7,110,0.963910,24.562851,6.539282
8,108,0.589867,13.703929,11.065927
9,87,0.885903,17.437609,8.088791


In [68]:
particle_df.to_csv('SEM_grain_metrics_016.csv', index=True, header=True)

## apply a physical size to the pixels... 


 just multiply the property by the pixel size and output to a csv

from header in image Image Pixel Size = ???? nm

In [ ]:
pix_size= 20

In [113]:
20**2

400

In [ ]:
maj_nm = pix_size* maj_ax
minor_nm = pix_size * min_ax

areas_nm = (pix_size**2)*areas

eqi_radius_nm = (equi_dia/2)*pix_size


In [ ]:
aspect_pix = maj_ax/min_ax
aspect_nm = maj_nm/minor_nm

In [ ]:
orient_deg=[]
for i in range(len(orient)):
    orient_deg.append(math.degrees(orient[i]))
#orient_deg    

In [ ]:
metrics_list =[part_id,corrs[: , 0],corrs[: , 1], areas, eccs,orient,orient_deg, maj_ax, min_ax, aspect_pix, maj_nm, minor_nm,aspect_nm]
#note skipping first (ie 0) element since that is the background
metrics_list= [[j[i] for j in metrics_list] for i in np.arange(1,len(metrics_list[0]))]

mets = np.array(metrics_list)
print(mets.shape)

met_titles=['ID','Row', 'Column','Areas', 'eccentricty', 'orientation (rad)','orientation (degree)','Major axis (pix)', 'Minor Axis (pix)','aspect ratio (pixel)','Major axis (nm)', 'Minor Axis (nm)','aspect ratio (nm)']

particle_df = pd.DataFrame.from_records(metrics_list, columns=met_titles)
particle_df.to_csv('SEM_grain_metrics_revised.csv', index=True, header=True)
particle_df


 <a href='#link2'> **Return to workflow**</a> 

### Appendix: gnerate markers - manual selection <a id='link10'></a>
to try and control where the seeds are a more time consuming appraoch is to manully place them.  need to use the qt4 enviroment too. Basically, this will pop up a window which you can click on to define where a particle is. The current code does not have a limit on particles or how long it takes. To 'end' the selction press the 'enter' key.{ Also look up the function documentation here: https://matplotlib.org/api/_as_gen/matplotlib.pyplot.ginput.html}


These points are then feed into the watershed used in section 3 to define foreground and background objects.  It does not help solve the pulling apart of grains problem however.

In [ ]:
%matplotlib qt4


In [ ]:
nlm.plot(cmap='viridis')

In [ ]:
grains3.plot()

In [22]:
from __future__ import print_function


plt.figure()

plt.imshow(nlm.data)
print("Please click")
markers = plt.ginput(-1,-1)
np.save('grain_markers', markers)
print("clicked", markers)
plt.show()

Please click
clicked [(46.7367858984116, 60.81480617680984), (51.45040875948246, 45.73121302138304), (64.64855277048088, 43.84576387695472), (83.50304421476443, 48.55938673802564), (107.07115852011879, 43.84576387695472), (124.98292539218818, 61.757530749024), (140.06651854761498, 42.90303930474056), (104.2429848034763, 73.07022561559415), (80.67487049812189, 74.01295018780831), (75.96124763705103, 86.26836962659263), (64.64855277048088, 81.55474676552171), (51.45040875948246, 95.69561534873435), (35.42409103184144, 105.12286107087607), (83.50304421476443, 128.69097537623054), (57.10675619276748, 126.8055262318021), (110.84205680897554, 96.63833992094851), (113.67023052561802, 115.49283136523206), (135.35289568654406, 105.12286107087607), (95.75846365354869, 132.46187366508718), (113.67023052561802, 145.66001767608566), (151.37921341418502, 113.60738222080374), (174.94732771953937, 138.11822109837226), (154.2073871308276, 149.4309159649423), (135.35289568654406, 131.51914909287302), (1

In [ ]:
clk_mark=markers

In [ ]:
%matplotlib inline

proably a better way to do this but

In [24]:
x, y = np.load('grain_markers.npy').T
plt.imshow(nlm.data, cmap='gray')
plt.plot(x, y, 'or', ms=4)

In [25]:
from skimage import morphology
seg_markers = np.zeros(grain_bin2.data.shape, dtype=np.int)
seg_markers[y.astype(np.int), x.astype(np.int)] = np.arange(len(x)) + 1
seg_markers = morphology.dilation(seg_markers, morphology.disk(3))
seg_markers_map=hs.signals.Signal2D(seg_markers)
seg_markers_map.change_dtype('float32')
seg_markers_map.metadata.General.title = 'clicked markers only'
seg_markers_map.data[seg_markers_map.data==0]=np.nan
seg_markers_map.plot(cmap='magma')




In [26]:
distance = ndi.distance_transform_edt(grains3.data)
labels = mph.watershed(grains3.data, seg_markers, mask=grains3.data)
grains_ws2= hs.signals.Signal2D(labels)
grains_ws2.change_dtype('float32')
grains_ws2.metadata.General.title = 'local maximum water shed'
grains_ws2.data[grains_ws2.data==0]=np.nan
grains_ws2.plot(cmap='magma')
num_grains = measure.label(labels, 
                        background=0)
print(num_grains.max())
num_grains = (num_grains + 1).astype(np.uint8)

605


In [40]:


labels = mph.watershed(-distance, seg_markers, mask=grains3.data)
grains_sep= hs.signals.Signal2D(labels)
grains_sep.change_dtype('float32')
grains_sep.metadata.General.title = 'h_maxima water shed'
grains_sep.data[grains_sep.data==0]=np.nan
grains_sep.plot(cmap='magma', scalebar=False)
num_grains = measure.label(labels, 
                        background=0)
print(num_grains.max())
num_grains = (num_grains + 1).astype(np.uint8)


605


 <a href='#link2'> **Return to workflow**</a>
 
 <a href='#link8'> ** object segmenting**</a>

In [ ]:
500*12

In [ ]:
(6000-4546)/500
